# Core concepts

## Predefined atlases, parcellations, and reference spaces

The `siibra.core` module provides classes for the core concepts of brain atlases. These include 

 - the `Atlas` class as the basic entry point for working with a brain atlas 
 - the `Parcellation` class, giving access to a particular brain segregation scheme
 - the `Space` class, givin access to a particular reference space
 
The above classes are all derived from the basic `AtlasConcept` class, and represent semantic concepts. They are special in the sense that `siibra` automatically builds a registry with predefined objects for each of them as soon as you import the package. To configure the predefined objects, siibra will retrieve configuration details from EBRAINS and some other online resources when you import the package: 

In [ ]:
import siibra

To performing this bootstrap process, an internet connection is required to import siibra the first time. However, the configuration information is then stored in a local cache folder on your system, so after the first time you will be able to import your `siibra` package without being online.

## Accessing predefined atlases

Siibra represents the core concepts via the registries  `siibra.atlases`, `siibra.parcellations`, and `siibra.spaces`. Elements in each registry can be accessed in different ways:

 - You can iterate over all objects
 - An integer index gives sequential access to individual elements
 - A string index will be matched against the name or key of objects. If it does not match exactly, an inexact string matching will be used to see if a unique entry can be found.
 - Object keys can be tab-completed as attributes of the registry

Let's try this out for the `siibra.atlases` registry.

In [ ]:
# Which atlases are provided? We can iterate over objects in the registry.
for atlas in siibra.atlases:
    print(atlas)

In [ ]:
# Access the first element in the registry
siibra.atlases[0]

In [ ]:
# Access elements by their name as attributes
# with autocompletion by most Python interpreters
siibra.atlases.MULTILEVEL_HUMAN_ATLAS

Note how both alternatives provide the exact same object. The easiest and recommended way to access items from a registry however, is to use keywords for accessing elements. `siibra` will try to figure out the matching item, or inform you if you need to be more precise.

In [ ]:
# The easiest way: Using string matching of keywords
siibra.atlases['human']